In [1]:
import requests
import pandas as pd
import numpy as np
import os

In [2]:
import simplejson as json

In [3]:
BEARER_TOKEN = os.environ['BEARER_TOKEN_TW']


## Get User

In [4]:
user_url = "https://api.twitter.com/2/users/by/username/btppubliceye"

payload={}
headers = {
  'Authorization': 'Bearer {}'.format(BEARER_TOKEN),
  
}


In [5]:
user_response = requests.request("GET", user_url, headers=headers, data=payload)
user_id = user_response.json()['data']['id']


In [6]:
def get_original_tweet(id):
    url = "https://api.twitter.com/2/tweets/{}".format(id[0]['id'])
    try:
        tweet = requests.request("GET", url, headers=headers, data=payload)
        return tweet.json()['data']['text']
    except:
        return ""

In [7]:
get_original_tweet([{'type': 'replied_to', 'id': '1348149446194864128'}])

'Police vehicle with dark tints. Taken today at 11:15 near Cauvery theatre. @blrcitytraffic @BlrCityPolice @btppubliceye https://t.co/sEnzcs1uo1'

## User Timeline

In [8]:
df =pd.DataFrame.from_dict({"in_reply_to_user_id":[],"text":[],"created_at":[],"id":[]})

In [9]:
def get_timeline_url(user_id,token):
    timeline_url =''
    if(len(token)>0):
        timeline_url="https://api.twitter.com/2/users/{}/tweets?expansions=attachments.media_keys&tweet.fields=in_reply_to_user_id,text,created_at,referenced_tweets&media.fields=preview_image_url,url&start_time=2020-12-01T00:00:01Z&end_time=2021-01-10T23:59:59Z&max_results=99".format(user_id)
        timeline_url=timeline_url+"&pagination_token={}".format(token)
    else:
        timeline_url="https://api.twitter.com/2/users/{}/tweets?expansions=attachments.media_keys&tweet.fields=in_reply_to_user_id,text,created_at,referenced_tweets&media.fields=preview_image_url,url&start_time=2020-12-01T00:00:01Z&end_time=2021-01-10T23:59:59Z&max_results=99".format(user_id)

    return timeline_url

In [10]:
i=0
token=''
response_json =[]
pagination_token=True
while(pagination_token==True):
    if i==0:
        timeline_url = get_timeline_url(user_id,'')
        timeline_response = requests.request("GET", timeline_url, headers=headers, data=payload)
        response_json.append(timeline_response.json())
        data = pd.json_normalize(response_json[0],record_path=['data'])
        df = pd.concat([df,data])
        if('meta' in timeline_response.json().keys()):
            print(timeline_response.json()['meta'])
            if 'next_token' in timeline_response.json()['meta'].keys():
                token = timeline_response.json()['meta']['next_token']
                i=1
            else:
                pagination_token=False
    else:
        timeline_url=get_timeline_url(user_id,token)
        timeline_response = requests.request("GET", timeline_url, headers=headers, data=payload)
        response_json.append(timeline_response.json())
        data = pd.json_normalize(response_json[-1],record_path=['data'])
        df = pd.concat([df,data])
        if('meta' in timeline_response.json().keys()):
            print(timeline_response.json()['meta'])
            if 'next_token' in timeline_response.json()['meta'].keys():
                token = timeline_response.json()['meta']['next_token']
                i=i+1
            else:
                pagination_token=False
        else:
                pagination_token=False
        print(timeline_response.json()['data'][-1]['created_at'])
           
        

{'oldest_id': '1335448320777457666', 'newest_id': '1348253930829078529', 'result_count': 99, 'next_token': '7140dibdnow9c7btw3w29iwb7bjfs44jf7wtx6i998njq'}
{'oldest_id': '1333704327093911555', 'newest_id': '1335223744361459712', 'result_count': 11, 'previous_token': '77qpymm88g5h9vqkluldpye2aemc1tcrhcoqk4cv897aq'}
2020-12-01T09:27:45.000Z


In [11]:
#with open('tweet.txt', 'w') as json_file:
#    json.dump(twitter_text_data, json_file)

In [12]:
df['original_tweet']=df['referenced_tweets'].apply(get_original_tweet)

In [13]:
df.head()

,in_reply_to_user_id,text,created_at,id,referenced_tweets,attachments.media_keys,original_tweet
0,1116569277199306753,@concern13161846 Please wait there is the serv...,2021-01-10T13:02:41.000Z,1348253930829078529,"[{'type': 'replied_to', 'id': '134814944619486...",NaN,Police vehicle with dark tints. Taken today at...
1,1116569277199306753,@concern13161846 @blrcitytraffic @BlrCityPolic...,2021-01-10T03:54:54.000Z,1348116077528580096,"[{'type': 'replied_to', 'id': '134792929685833...",NaN,Looks like police have forgotten that they nee...
2,1340848044560982016,@BangaloreRavi1 please mention exact location.,2021-01-09T13:24:46.000Z,1347897100181204993,"[{'type': 'replied_to', 'id': '134789509558490...",NaN,@btppubliceye Of late many street vendors are ...
3,219674551,"@arunchid Noted, have informed to technical te...",2021-01-09T13:23:29.000Z,1347896780772442117,"[{'type': 'replied_to', 'id': '134788318393355...",NaN,@btppubliceye \n\nThe signal near navarang bar...
4,223543322,@intrepidcritter Please provide exact date and...,2021-01-09T07:56:21.000Z,1347814454046515202,"[{'type': 'replied_to', 'id': '134777172921765...",NaN,"@btppubliceye Chennamana Kere achhkattu, BSK 3..."


In [14]:
df.to_excel('btp-01122020T10012021.xlsx',index=False)

In [15]:
df.count()

in_reply_to_user_id       108
text                      110
created_at                110
id                        110
referenced_tweets         110
attachments.media_keys      2
original_tweet            110
dtype: int64